In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
!pip install -qU pinecone-client pinecone-datasets langchain-pinecone

In [ ]:
!pip install --quiet langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python langchain-mistralai gpt4all

In [36]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.chat_models import ChatOllama
from langchain.load import dumps, loads
from operator import itemgetter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
import pinecone
from pinecone import Pinecone, ServerlessSpec, PodSpec
from langchain.prompts import ChatPromptTemplate
from google.colab import userdata
import time
from langchain import hub
import json

In [5]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
pinecone_api_key = os.environ['PINECONE_API_KEY']
use_serverless = True

In [6]:
# Load blog
loader = WebBaseLoader(
    web_paths=( "https://en.wikipedia.org/wiki/Albert_Einstein",
                "https://www.nobelprize.org/prizes/physics/1921/einstein/biographical/",
                "https://www.britannica.com/biography/Albert-Einstein/General-relativity-and-teaching-career",
                "https://www.space.com/15524-albert-einstein.html"),
    # bs_kwargs=dict(
    #     parse_only=bs4.SoupStrainer(
    #         class_=("post-content", "post-title", "post-header")
    #     )
    # ),
)
blog_docs = loader.load()

In [7]:
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [8]:
len(splits)

932

In [9]:
pc = Pinecone(api_key=pinecone_api_key)
if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-west-2')
else:
    # if not using a starter index, you should specify a pod_type too
    spec = PodSpec()
# check for and delete index if already exists
index_name = 'einstein'
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)
# create a new index
pc.create_index(
    index_name,
    dimension=384,  # dimensionality of GPT4ALLEmbeddings
    metric='dotproduct',
    spec=spec
)
# wait for index to be initialized
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [10]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [11]:
from langchain.vectorstores import Pinecone
embedding = GPT4AllEmbeddings()
vectorstore = Pinecone.from_documents(splits, embedding, index_name = index_name)
retriever = vectorstore.as_retriever()

Downloading: 100%|██████████| 45.9M/45.9M [00:00<00:00, 64.3MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 551MiB/s]


In [12]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [13]:
# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of 3 sub-problems / sub-questions that can be answers in isolation. \n
Generate 3 search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [18]:
# LLM
local_llm = "mistral-openorca"

llm = ChatOllama(model=local_llm, format="json", temperature=0.75)

# Chain
generate_queries_decomposition = (  prompt_decomposition
                                    | llm
                                    | StrOutputParser()
                                    # | (lambda x: x.values())
                                  )

# Run
question = "What is Albert Einstein most famous for"
questions = generate_queries_decomposition.invoke({"question":question})
questions

'{\n  "1": "What was Albert Einstein\'s Theory of Relativity?",\n  "2": "Who is Albert Einstein?",\n  "3": "What did Albert Einstein contribute to modern physics?"\n}<|im_end|>'

In [20]:
questions_formatted = questions.split("\n")[1:4]
questions_final = [q.split(":")[1].strip().split(",")[0][1:-1] for q in questions_formatted]

In [21]:
questions_final

["What was Albert Einstein's Theory of Relativity?",
 'Who is Albert Einstein?',
 'What did Albert Einstein contribute to modern physics?']

In [22]:
template = """Here is the question you need to answer:

\n --- \n {question} \n --- \n

Here is any available background question + answer pairs:

\n --- \n {q_a_pairs} \n --- \n

Here is additional context relevant to the question:

\n --- \n {context} \n --- \n

Use the above context and any background question + answer pairs to answer the question: \n {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [23]:
def format_qa_pair(question, answer):
    """Format Q and A pair"""

    formatted_string = ""
    formatted_string += f"Question: {question}\nAnswer: {answer}\n\n"
    return formatted_string.strip()


q_a_pairs = ""
for q in questions_final:

    rag_chain = (
    {   "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "q_a_pairs": itemgetter("q_a_pairs")}
    | decomposition_prompt
    | llm
    | StrOutputParser())

    answer = rag_chain.invoke({"question":q,"q_a_pairs":q_a_pairs})
    q_a_pair = format_qa_pair(q,answer)
    q_a_pairs = q_a_pairs + "\n---\n"+  q_a_pair

In [34]:
json.loads(answer[:-10])

{'answers': [{'score': 1,
   'content': "Albert Einstein contributed significantly to modern physics by developing the theory of relativity. His theories revolutionized our understanding of space, time, and gravity. The special theory of relativity focuses on the relationship between space and time in constant velocity systems, while the general theory of relativity explains the effects of gravity on objects in motion. Einstein's mass-energy equivalence formula (E=mc^2) further changed our comprehension of energy and matter."}]}